In [1]:
from IPython.utils import io
import tqdm.notebook
import os, sys, random
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
      # Instalar rdkit
      !pip -q install rdkit
      pbar.update(25)
      # Instalar molplotly
      !pip install molplotly
      pbar.update(50)
      # Instalar jupyter-dash
      !pip install jupyter-dash
      pbar.update(75)
      # Instalar el diseño de aplicación dash
      !pip install dash-bootstrap-components
      pbar.update(100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [18]:
from IPython.utils import io
import tqdm.notebook
import os, sys, random

In [3]:
import rdkit
print(f"rdkit_version: {rdkit.__version__}")

rdkit_version: 2022.09.5


In [4]:
# Importar blibliotecas
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys, AllChem
from scipy.spatial.distance import pdist, squareform

### 1. Load dataset

In [5]:
# Fragmentos
url_data = "https://drive.google.com/file/d/1BOBSUqrWCHBcpfkrrSoQT_JfXkAvrkzV/view?usp=drive_link"
url_data='https://drive.google.com/uc?id=' + url_data.split('/')[-2]
lanapdb = pd.read_csv(url_data)
lanapdb = lanapdb[["ID", "SMILES_chiral"]]
lanapdb["Database"] = "LANaPDB"
lanapdb.tail(2)

,ID,SMILES_chiral,Database
1830,LANaPDB_fragments_73605,CC1(C)[C@H]2CCC[C@]3(CO3)[C@]2(C)C[C@H](O)[C@]...,LANaPDB
1831,LANaPDB_fragments_74033,CCC(C)=CC(=O)O,LANaPDB


In [6]:
# Fragmentos
url_data_COCONUT = "https://drive.google.com/file/d/1XM0rEgSrUBZ_Rc6M29zpGtHyFipiy0vz/view?usp=drive_link"
url_data_COOCONUT ='https://drive.google.com/uc?id=' + url_data_COCONUT.split('/')[-2]
coconut = pd.read_csv(url_data_COOCONUT)
coconut = coconut[["ID", "SMILES_chiral"]]
coconut["Database"] = "COCONUT"
coconut.tail(2)

,ID,SMILES_chiral,Database
38745,COCONUT_fragments_2583030,O=c1cco[nH]1,COCONUT
38746,COCONUT_fragments_2583083,O=CC1C=CCC1,COCONUT


In [7]:
# Fragmentos
url_data_EnamineSolWat = "https://drive.google.com/file/d/1-Oo0_FgEcgkHkjrpoGOM5voQSc3-G-jd/view?usp=sharing"
url_data_EnamineSolWat ='https://drive.google.com/uc?id=' + url_data_EnamineSolWat.split('/')[-2]
Enamine = pd.read_csv(url_data_EnamineSolWat)
Enamine = Enamine[["ID", "SMILES_chiral"]]
Enamine["Database"] = "Enamine"
Enamine.tail(2)

,ID,SMILES_chiral,Database
8384,Z212848872,CC1CN(C(=O)Cc2cccc(F)c2)CCO1,Enamine
8385,Z228588126,CC(C(=O)N1CCCCCC1)N1CCNCC1,Enamine


In [8]:
# Fragmentos
url_data_ChemDiv = "https://drive.google.com/file/d/1-4btTI1dpHuZUAdDBBVXNg6HJcwaFaBL/view?usp=sharing"
url_data_ChemDiv ='https://drive.google.com/uc?id=' + url_data_ChemDiv.split('/')[-2]
ChemDiv = pd.read_csv(url_data_ChemDiv)
ChemDiv = ChemDiv[["ID", "SMILES_chiral"]]
ChemDiv["Database"] = "ChemDiv"
ChemDiv.tail(2)

,ID,SMILES_chiral,Database
16721,BB01-0446,CC(C)N1CCC2(CC1)NC(=O)c1ccccc1O2,ChemDiv
16722,BB01-0451,CC(=O)N1C(C(=O)O)CSC1c1ccc(C)cc1,ChemDiv


In [9]:
# Fragmentos
url_data_Maybridge = "https://drive.google.com/file/d/1-A5PY17BgEWeDHHhAa_E711LV3ZtwO74/view?usp=sharing"
url_data_Maybridge='https://drive.google.com/uc?id=' + url_data_Maybridge.split('/')[-2]
Maybridge = pd.read_csv(url_data_Maybridge)
Maybridge = Maybridge[["ID", "SMILES_chiral"]]
Maybridge["Database"] = "Maybridge"
Maybridge.tail(2)

,ID,SMILES_chiral,Database
5910,KM08807,O=C1CC2CCCC(C1)N2Cc1ccccc1,Maybridge
5911,SEW04324,O=C(c1ccccc1)C1CCN(C(=O)C(F)(F)F)CC1,Maybridge


In [10]:
# Fragmentos
url_data_LifeChemicals = "https://drive.google.com/file/d/1-LXu2oQZLMmaPIttlCQnljSIABAaYe9g/view?usp=sharing"
url_data_LifeChemicals ='https://drive.google.com/uc?id=' + url_data_LifeChemicals.split('/')[-2]
LifeChemicals = pd.read_csv(url_data_LifeChemicals)
LifeChemicals = LifeChemicals[["ID", "SMILES_chiral"]]
LifeChemicals["Database"] = "Life Chemicals"
LifeChemicals.tail(2)

,ID,SMILES_chiral,Database
14732,F6782-0679,Cn1cc(C(=O)c2ccccc2)c(=O)c2cc(F)ccc21,Life Chemicals
14733,F0788-0005,c1cc[n+]2cc3n(c2c1)Cc1c[n+]2ccccc2n1C3,Life Chemicals


In [11]:
# Fragmentos
url_data_CRAFT = "https://drive.google.com/file/d/1-4BEojtFtevY9nSlEkKLzQ5tN3PpLqrE/view?usp=sharing"
url_data_CRAFT ='https://drive.google.com/uc?id=' + url_data_CRAFT.split('/')[-2]
CRAFT = pd.read_csv(url_data_CRAFT)
CRAFT = CRAFT[["ID", "SMILES_chiral"]]
CRAFT["Database"] = "CRAFT"
CRAFT.tail(2)

,ID,SMILES_chiral,Database
174,QHM-0000005,O=C1C=C(Cl)C(=O)c2ccccc21,CRAFT
175,QHM-0000002,CC1(C)CCC2=C(O1)c1ccccc1C(=O)C2=O,CRAFT


In [12]:
DATA = pd.concat([coconut, LifeChemicals, Enamine, ChemDiv, Maybridge, lanapdb, CRAFT])
DATA.head(2)

,ID,SMILES_chiral,Database
0,COCONUT_fragments_1,CC=O,COCONUT
1,COCONUT_fragments_2,CCCO,COCONUT


In [13]:
set(list(DATA["Database"]))

{'COCONUT',
 'CRAFT',
 'ChemDiv',
 'Enamine',
 'LANaPDB',
 'Life Chemicals',
 'Maybridge'}

### MACCS keys

---



In [14]:
# Check for invalid SMILES strings
data1 = DATA.copy()
#DATA = 0
invalid_smiles = [x for x in data1['SMILES_chiral'] if Chem.MolFromSmiles(x) is None]

# Print the invalid SMILES strings
print(f"Invalid SMILES strings: {invalid_smiles}")

# Remove the invalid SMILES strings from the DataFrame
data1 = data1[~data1['SMILES_chiral'].isin(invalid_smiles)]

# Recalculate the MACCS keys
fps = [[int(y) for y in MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(x)).ToBitString()] for x in data1['SMILES_chiral']]
fps = pd.DataFrame([np.array(x) for x in fps])
fps

Invalid SMILES strings: []


,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86505,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,1,1,1,1,0
86506,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,1,1,1,1,0
86507,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,0,1,0
86508,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,1,1,0


In [15]:
#Data Normalized
from sklearn.preprocessing import StandardScaler
data_std = fps.copy()
data_std = StandardScaler().fit_transform(data_std)

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(data_std)
pca_results

# Complementary information
label = data1[["Database", "ID", "SMILES_chiral"]]
label = label.to_numpy()

# Concat arrays de numpy
arr = np.concatenate((label, pca_results), axis = 1)
# Crreate a new dataframe
pca_dataset = pd.DataFrame(data=arr, columns = ['Database',"ID", 'SMILES_chiral','component1', 'component2'])
pca_dataset.head(2)

# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

array([0.10308051, 0.06672785])

In [16]:
pca_dataset.head(2)

,Database,ID,SMILES_chiral,component1,component2
0,COCONUT,COCONUT_fragments_1,CC=O,-2.705999,1.110709
1,COCONUT,COCONUT_fragments_2,CCCO,-3.627726,0.246843


In [17]:
# Graficar
import plotly.express as px
import molplotly
from dash import Dash, dcc, html

color = [
         "turquoise", #COCONUT
         "crimson", #Life Chemicals
         "black", #Enamine
         "royalblue", #ChemDiv
         "mediumorchid", #Maybridge
         "green", #LANaPDB
         "orange", #CRAFT
        ]
fig_pca = px.scatter(pca_dataset,
                            x='component1',
                            y='component2',
                            color='Database',
                            color_discrete_sequence= color,
                            title='PCA: MACCS Fragments RO3',
                            labels={'component1': f'PC1 {round(exp_var_pca[0] * 100, 2)}%',
                                    'component2': f'PC2 {round(exp_var_pca[1] * 100, 2)}%'},
                            width=700,
                            height=500)

app_marker = molplotly.add_molecules(fig=fig_pca,
                                         df=pca_dataset,
                                         smiles_col='SMILES_chiral',
                                         title_col='ID',
                                         color_col="Database", #'Data_set_LOGBB_EXP'
                                        )
#fig_tsne.show()
app_marker.run(port=8060)

/usr/local/lib/python3.11/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

## Morgan2

---



In [ ]:
# Check for invalid SMILES strings
invalid_smiles = [x for x in data1['SMILES_chiral'] if Chem.MolFromSmiles(x) is None]

# Print the invalid SMILES strings
print(f"Invalid SMILES strings: {invalid_smiles}")

# Remove the invalid SMILES strings from the DataFrame
data1 = data1[~data1['SMILES_chiral'].isin(invalid_smiles)]

# Recalculate the Morgan2
fps = [[int(y) for y in AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x), 2, nBits=1024).ToBitString()] for x in data1['SMILES_chiral']]
fps = pd.DataFrame([np.array(x) for x in fps])
fps

Invalid SMILES strings: []


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86505,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
86506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86508,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Data Normalized
from sklearn.preprocessing import StandardScaler
data_std = fps.copy()
data_std = StandardScaler().fit_transform(data_std)

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(data_std)
pca_results

# Complementary information
label = data1[["Database", "ID", "SMILES_chiral"]]
label = label.to_numpy()

# Concat arrays de numpy
arr = np.concatenate((label, pca_results), axis = 1)
# Crreate a new dataframe
pca_dataset = pd.DataFrame(data=arr, columns = ['Database',"ID", 'SMILES_chiral','component1', 'component2'])
pca_dataset.head(2)

# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

array([0.00873664, 0.00627085])

In [ ]:
# Graficar
import plotly.express as px
import molplotly
from dash import Dash, dcc, html

color = [
         "turquoise", #COCONUT
         "crimson", #Life Chemicals
         "black", #Enamine
         "royalblue", #ChemDiv
         "mediumorchid", #Maybridge
         "green", #LANaPDB
         "orange", #CRAFT
        ]
fig_pca = px.scatter(pca_dataset,
                            x='component1',
                            y='component2',
                            color='Database',
                            color_discrete_sequence= color,
                            title='PCA: Morgan2 Fragments RO3',
                            labels={'component1': f'PC1 {round(exp_var_pca[0] * 100, 2)}%',
                                    'component2': f'PC2 {round(exp_var_pca[1] * 100, 2)}%'},
                            width=700,
                            height=500)

app_marker = molplotly.add_molecules(fig=fig_pca,
                                         df=pca_dataset,
                                         smiles_col='SMILES_chiral',
                                         title_col='ID',
                                         color_col="Database", #'Data_set_LOGBB_EXP'
                                        )
#fig_tsne.show()
app_marker.run(port=8060)

/usr/local/lib/python3.10/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

### Morgan3

---



In [ ]:
# Check for invalid SMILES strings
invalid_smiles = [x for x in data1['SMILES_chiral'] if Chem.MolFromSmiles(x) is None]

# Print the invalid SMILES strings
print(f"Invalid SMILES strings: {invalid_smiles}")

# Remove the invalid SMILES strings from the DataFrame
data1 = data1[~data1['SMILES_chiral'].isin(invalid_smiles)]

# Recalculate the Morgan2
fps = [[int(y) for y in AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x), 3, nBits=1024).ToBitString()] for x in data1['SMILES_chiral']]
fps = pd.DataFrame([np.array(x) for x in fps])
fps

Invalid SMILES strings: []


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86505,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
86506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
86507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86508,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Data Normalized
from sklearn.preprocessing import StandardScaler
data_std = fps.copy()
data_std = StandardScaler().fit_transform(data_std)

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(data_std)
pca_results

# Complementary information
label = data1[["Database", "ID", "SMILES_chiral"]]
label = label.to_numpy()

# Concat arrays de numpy
arr = np.concatenate((label, pca_results), axis = 1)
# Crreate a new dataframe
pca_dataset = pd.DataFrame(data=arr, columns = ['Database',"ID", 'SMILES_chiral','component1', 'component2'])
pca_dataset.head(2)

# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

array([0.00780633, 0.00627036])

In [ ]:
# Graficar
import plotly.express as px
import molplotly
from dash import Dash, dcc, html

color = [
         "turquoise", #COCONUT
         "crimson", #Life Chemicals
         "black", #Enamine
         "royalblue", #ChemDiv
         "mediumorchid", #Maybridge
         "green", #LANaPDB
         "orange", #CRAFT
        ]
fig_pca = px.scatter(pca_dataset,
                            x='component1',
                            y='component2',
                            color='Database',
                            color_discrete_sequence= color,
                            title='PCA: Morgan3 Fragments RO3',
                            labels={'component1': f'PC1 {round(exp_var_pca[0] * 100, 2)}%',
                                    'component2': f'PC2 {round(exp_var_pca[1] * 100, 2)}%'},
                            width=700,
                            height=500)

app_marker = molplotly.add_molecules(fig=fig_pca,
                                         df=pca_dataset,
                                         smiles_col='SMILES_chiral',
                                         title_col='ID',
                                         color_col="Database", #'Data_set_LOGBB_EXP'
                                        )
#fig_tsne.show()
app_marker.run(port=8060)

/usr/local/lib/python3.10/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>